In [1]:
import pandas as pd
# library for train test split
from sklearn.model_selection import train_test_split

# deep learning libraries for text pre-processing
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Modeling 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers, models, regularizers,initializers
from sklearn.model_selection import StratifiedKFold 
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding

In [2]:
df = pd.read_csv('df_final.csv')

In [3]:
df

,Unnamed: 0,reviewId,content,label,Target_type
0,0,fd212db8-ac94-4b33-9321-5e123f8686cd,saya suka dengan aplikasi ini benar benar memu...,negatif,0
1,1,b8cc5911-6498-4448-8a12-184b444162fc,halo saya pengguna lama dari tahun awalnya say...,positif,1
2,2,5132d329-c7ca-44a0-9975-750c161558e3,sejak tanggal juni saya transaksi flip ke btn ...,negatif,0
3,3,1d926f7b-902d-4f21-9858-554947e5ab3a,sekarang lebih ribet kalau mau validasi status...,negatif,0
4,4,578e21b8-85f6-43b7-9ee7-b98096581a8f,aplikasi nya bagus sih hanya sayang fitur prin...,positif,1
...,...,...,...,...,...
984,984,a75b9302-989c-476b-9a99-00d8da418dd1,tranfer biaya admin termurah,positif,1
985,985,44674d32-5b6e-4af9-8bc9-934a8307ea42,bagus semua transaksi untuk beli pulsa listrik...,positif,1
986,986,130aec02-cd80-4659-b428-20dcc421c82a,aplikasi terbaik sangat fungsional bikin kanto...,positif,1
987,987,cfae2f94-38bf-4e6e-ba8e-18395f433024,selalu lancar dan terbaik lah,positif,1


In [4]:
X = df['content'].values
y = df['Target_type'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [5]:
NUM_WORDS = 15000

tokenizer = Tokenizer(num_words=NUM_WORDS, split=' ')
tokenizer.fit_on_texts(X_train)

In [6]:
x_train_token = tokenizer.texts_to_sequences(X_train)
x_test_token = tokenizer.texts_to_sequences(X_test)

In [7]:
MAXLEN = 250

x_train_padding = pad_sequences(x_train_token, maxlen=MAXLEN, padding='pre', truncating='post')
x_test_padding = pad_sequences(x_test_token, maxlen=MAXLEN, padding='pre', truncating='post')
embeding_dim = 16

In [81]:
model = Sequential([
     Embedding(input_dim=NUM_WORDS, input_length=MAXLEN, output_dim=16),
    tf.keras.layers.Bidirectional(LSTM(100, return_sequences=True)),
    tf.keras.layers.Bidirectional(LSTM(50)),
    Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 250, 16)           240000    
                                                                 
 bidirectional_2 (Bidirectio  (None, 250, 200)         93600     
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 100)              100400    
 nal)                                                            
                                                                 
 dense_16 (Dense)            (None, 16)                1616      
                                                                 
 dropout_8 (Dropout)         (None, 16)                0         
                                                                 
 dense_17 (Dense)            (None, 1)                

In [82]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

In [83]:
class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')> 0.95):
      print("\nAkurasi validasi telah mencapai >95%!")
      self.model.stop_training = True
callbacks = MyCallback()

In [84]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [85]:
EPOCHS = 20
BATCH_SIZE = 30
history = model.fit(x_train_padding,
                    y_train,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    validation_data=(x_test_padding, y_test),
                    verbose=1,
                    callbacks = [early_stopping,callbacks])

Epoch 1/20
27/27 [==============================] - 9s 107ms/step - loss: 0.6585 - accuracy: 0.6599 - val_loss: 0.6346 - val_accuracy: 0.7172
Epoch 2/20
27/27 [==============================] - 1s 44ms/step - loss: 0.6118 - accuracy: 0.6726 - val_loss: 0.5088 - val_accuracy: 0.7778
Epoch 3/20
27/27 [==============================] - 1s 45ms/step - loss: 0.4749 - accuracy: 0.8066 - val_loss: 0.3681 - val_accuracy: 0.8384
Epoch 4/20
27/27 [==============================] - 1s 43ms/step - loss: 0.3201 - accuracy: 0.9001 - val_loss: 0.3259 - val_accuracy: 0.8687
Epoch 5/20
27/27 [==============================] - 1s 44ms/step - loss: 0.2124 - accuracy: 0.9279 - val_loss: 0.3328 - val_accuracy: 0.8586
Epoch 6/20
27/27 [==============================] - ETA: 0s - loss: 0.1392 - accuracy: 0.9558
Akurasi validasi telah mencapai >95%!
27/27 [==============================] - 1s 45ms/step - loss: 0.1392 - accuracy: 0.9558 - val_loss: 0.3715 - val_accuracy: 0.8636
